In [1]:
# import cv2
# import numpy as np

# # Initialize the webcam
# cap = cv2.VideoCapture(0)

# # Define the aspect ratio for a typical face
# EXPECTED_ASPECT_RATIO = 1.6  # Height/Width ratio

# while True:
#     # Capture frame-by-frame
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Convert to grayscale
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     # Apply Gaussian blur to reduce noise
#     blurred = cv2.GaussianBlur(gray, (5, 5), 1.6)

#     # Perform Canny edge detection
#     edges = cv2.Canny(blurred, threshold1=100, threshold2=200)

#     # Find contours in the edge-detected image
#     contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     # Iterate through each contour
#     for contour in contours:
#         # Fit an ellipse to the contour if it has enough points
#         if len(contour) >= 5:
#             ellipse = cv2.fitEllipse(contour)
#             center, axes, angle = ellipse
#             major_axis, minor_axis = max(axes), min(axes)
#             aspect_ratio = major_axis / (minor_axis + 0.00000001)

#             # Check if the aspect ratio is within a reasonable range
#             if 1.4 < aspect_ratio < 1.8:
#                 # Draw the ellipse on the original frame
#                 cv2.ellipse(frame, ellipse, (0, 255, 0), 2)

#                 # Optionally, draw the center and axes
#                 cv2.circle(frame, (int(center[0]), int(center[1])), 5, (0, 0, 255), -1)
#                 cv2.line(frame, (int(center[0] - axes[0] / 2), int(center[1])), 
#                          (int(center[0] + axes[0] / 2), int(center[1])), (255, 0, 0), 2)
#                 cv2.line(frame, (int(center[0]), int(center[1] - axes[1] / 2)), 
#                          (int(center[0]), int(center[1] + axes[1] / 2)), (255, 0, 0), 2)

#     # Display the resulting frame
#     cv2.imshow('Real-Time Face Detection', frame)

#     # Break the loop on 'q' key press
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the webcam and close all OpenCV windows
# cap.release()
# cv2.destroyAllWindows()



In [2]:
import os
import cv2
import dlib
import numpy as np
import matplotlib.pyplot as plt

# Dataset paths
dataset_path = "/content/face_dataset/merged/images/train"
labels_path = "/content/face_dataset/merged/labels/train"

# Process a smaller subset first
MAX_IMAGES = 100  # Change as needed
image_count = 0  # Counter for processed images

face_size = (64, 64)
faces = []
labels = []
detected_images_list = []  # Store images with bounding boxes for visualization
missing_labels = 0

# Load HOG + SVM-based face detector from dlib
hog_detector = dlib.get_frontal_face_detector()

# Pre-load label filenames into a dictionary for fast lookup
label_files = {os.path.splitext(f)[0]: f for f in os.listdir(labels_path)}

# Iterate over all image files
for image_name in os.listdir(dataset_path):
    if image_count >= MAX_IMAGES:
        break  # Stop processing after reaching MAX_IMAGES
    image_count += 1

    image_path = os.path.join(dataset_path, image_name)

    # Read image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Skipping unreadable image: {image_path}")
        continue

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)  # Improves contrast

    # Detect faces using HOG + SVM
    detected_faces = hog_detector(gray)

    if len(detected_faces) == 0:
        continue  # Skip images with no faces

    # Draw bounding boxes
    for face in detected_faces:
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green box around detected face

    # Store the image with bounding boxes
    detected_images_list.append(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Convert for Matplotlib display

    # Match labels faster
    base_name = os.path.splitext(image_name)[0]  # Remove file extension
    label_filename = label_files.get(base_name, None)  # Find label file

    if label_filename:
        label_path = os.path.join(labels_path, label_filename)
        with open(label_path, "r") as label_file:
            label = label_file.read().strip()
            labels.append(label)
    else:
        missing_labels += 1
        labels.append("Unknown")

# Convert to NumPy arrays
faces = np.array(faces)
labels = np.array(labels)

print(f"Total faces detected: {len(detected_images_list)}")
print(f"Total missing labels: {missing_labels}")  # Print at the end

# 📌 Display multiple images in 2-3 rows with bounding boxes
num_images_to_display = min(9, len(detected_images_list))  # Display up to 9 images
rows = 3
cols = num_images_to_display // rows if num_images_to_display % rows == 0 else (num_images_to_display // rows) + 1

fig, axes = plt.subplots(rows, cols, figsize=(15, 10))

# Flatten the axes array for easier indexing
axes = axes.flatten()

for i in range(num_images_to_display):
    axes[i].imshow(detected_images_list[i])  # Show image with bounding box
    axes[i].axis("off")
    axes[i].set_title(f"Image {i+1}")

plt.tight_layout()
plt.show()


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/face_dataset/merged/labels/train'

In [ ]:
import cv2
import numpy as np

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Define the aspect ratio for a typical face
EXPECTED_ASPECT_RATIO = 1.6  # Height/Width ratio

# Initialize OpenCV's Kalman Filter for tracking
kalman = cv2.KalmanFilter(4, 2)  # 4 state variables (x, y, dx, dy) and 2 measurement variables (x, y)
kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], dtype=np.float32)
kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], dtype=np.float32)
kalman.processNoiseCov = np.array([[1e-2, 0, 0, 0], [0, 1e-2, 0, 0], [0, 0, 1e-5, 0], [0, 0, 0, 1e-5]], dtype=np.float32)
kalman.measurementNoiseCov = np.array([[1, 0], [0, 1]], dtype=np.float32)

# Previous frame for motion detection
prev_gray = None

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Convert to YCbCr color space for skin segmentation
    ycbcr = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)

    # Define skin color range
    lower_skin = np.array([0, 133, 77], dtype=np.uint8)
    upper_skin = np.array([255, 173, 127], dtype=np.uint8)

    # Skin mask using the defined range
    skin_mask = cv2.inRange(ycbcr, lower_skin, upper_skin)

    # Apply the mask to the frame
    skin = cv2.bitwise_and(frame, frame, mask=skin_mask)

    # Perform frame differencing for motion detection (if previous frame exists)
    if prev_gray is not None:
        frame_diff = cv2.absdiff(prev_gray, gray)
        _, thresh = cv2.threshold(frame_diff, 50, 255, cv2.THRESH_BINARY)

        # Find contours in the thresholded difference image (motion regions)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            if len(contour) >= 5:
                ellipse = cv2.fitEllipse(contour)
                center, axes, angle = ellipse
                major_axis, minor_axis = max(axes), min(axes)
                aspect_ratio = major_axis / (minor_axis + 0.00000001)
                # Eccentricity of the ellipse for face validation
                eccentricity = np.sqrt(1 - (minor_axis / major_axis) ** 2)

                # Check if the contour fits within the expected face shape
                if 1.4 < aspect_ratio < 1.8 and eccentricity < 0.9:
                    cv2.ellipse(frame, ellipse, (0, 255, 0), 2)
                    # Kalman filter tracking for predicted face position
                    measurement = np.array([[np.float32(center[0])], [np.float32(center[1])]])
                    kalman.correct(measurement)
                    prediction = kalman.predict()

                    predicted_center = (int(prediction[0]), int(prediction[1]))
                    cv2.circle(frame, predicted_center, 5, (0, 0, 255), -1)

    # Store the current grayscale frame for the next iteration
    prev_gray = gray

    # Display the resulting frame
    cv2.imshow('Real-Time Face Detection', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

: 

In [ ]:
import cv2
import numpy as np

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Initialize OpenCV's HOG + SVM face detector
hog_face_detector = cv2.HOGDescriptor()
hog_face_detector.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Initialize OpenCV's Kalman Filter for tracking
kalman = cv2.KalmanFilter(4, 2)  # 4 state variables (x, y, dx, dy) and 2 measurement variables (x, y)
kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], dtype=np.float32)
kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], dtype=np.float32)
kalman.processNoiseCov = np.array([[1e-2, 0, 0, 0], [0, 1e-2, 0, 0], [0, 0, 1e-5, 0], [0, 0, 0, 1e-5]], dtype=np.float32)
kalman.measurementNoiseCov = np.array([[1, 0], [0, 1]], dtype=np.float32)

# Previous frame for motion detection
prev_gray = None

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # HOG face detection (alternative method)
    faces, _ = hog_face_detector.detectMultiScale(gray, winStride=(8, 8), padding=(8, 8), scale=1.05)

    # Draw bounding boxes for faces detected using HOG + SVM
    for (x, y, w, h) in faces:
        print(x,y,w,h)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Optionally, apply Kalman Filter for tracking detected faces (only for motion-based tracking)
    if prev_gray is not None:
        frame_diff = cv2.absdiff(prev_gray, gray)
        _, thresh = cv2.threshold(frame_diff, 50, 255, cv2.THRESH_BINARY)

        # Find contours in the thresholded difference image (motion regions)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            if len(contour) >= 5:
                ellipse = cv2.fitEllipse(contour)
                center, axes, angle = ellipse
                major_axis, minor_axis = max(axes), min(axes)
                aspect_ratio = major_axis / (minor_axis + 0.00000000001)
                # Eccentricity of the ellipse for face validation
                eccentricity = np.sqrt(1 - (minor_axis / major_axis) ** 2)

                # Check if the contour fits within the expected face shape
                if 1.4 < aspect_ratio < 1.8 and eccentricity < 0.9:
                    cv2.ellipse(frame, ellipse, (0, 255, 0), 2)
                    # Kalman filter tracking for predicted face position
                    measurement = np.array([[np.float32(center[0])], [np.float32(center[1])]])
                    kalman.correct(measurement)
                    prediction = kalman.predict()

                    predicted_center = (int(prediction[0]), int(prediction[1]))
                    cv2.circle(frame, predicted_center, 5, (0, 0, 255), -1)

    # Store the current grayscale frame for the next iteration
    prev_gray = gray

    # Display the resulting frame with bounding boxes
    cv2.imshow('Real-Time Face Detection', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Initialize OpenCV's HOG + SVM face detector
hog_face_detector = cv2.HOGDescriptor()
hog_face_detector.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Initialize Local Binary Pattern Histogram (LBPH) Face Recognizer
lbph_recognizer = cv2.face.LBPHFaceRecognizer_create()

# Load pre-trained LBPH model (if available)
try:
    lbph_recognizer.read("lbph_face_model.xml")  # Ensure you have a trained model
    print("LBPH model loaded successfully.")
except:
    print("No LBPH model found. Only detection will work.")

# Initialize Kalman Filter for face tracking
kalman = cv2.KalmanFilter(4, 2)  # 4 state variables (x, y, dx, dy), 2 measurements (x, y)
kalman.transitionMatrix = np.array([[1, 0, 1, 0], 
                                    [0, 1, 0, 1], 
                                    [0, 0, 1, 0], 
                                    [0, 0, 0, 1]], dtype=np.float32)
kalman.measurementMatrix = np.array([[1, 0, 0, 0], 
                                     [0, 1, 0, 0]], dtype=np.float32)
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 1e-2
kalman.measurementNoiseCov = np.eye(2, dtype=np.float32) * 1

# Variables for tracking
tracking = False
tracked_bbox = None

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # HOG-SVM Face Detection
    faces, _ = hog_face_detector.detectMultiScale(gray, winStride=(8, 8), padding=(8, 8), scale=1.05)

    # If a face is detected, update the Kalman filter
    if len(faces) > 0:
        tracking = True  # Start tracking when a face is detected
        x, y, w, h = faces[0]  # Take the first detected face

        # Update Kalman Filter with the detected bounding box center
        center_x, center_y = x + w // 2, y + h // 2
        measurement = np.array([[np.float32(center_x)], [np.float32(center_y)]])
        kalman.correct(measurement)

        # Save detected face coordinates
        tracked_bbox = (x, y, w, h)

        # Recognize face with LBPH (if trained)
        face_roi = gray[y:y+h, x:x+w]
        try:
            label, confidence = lbph_recognizer.predict(face_roi)
            text = f"ID: {label} | Conf: {round(confidence, 2)}"
            cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        except:
            pass  # If no trained model, skip recognition

        # Draw bounding box around the detected face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    if tracking and tracked_bbox:
        # Predict next position of the face
        prediction = kalman.predict()
        predicted_x, predicted_y = int(prediction[0]), int(prediction[1])

        # Draw the predicted position
        cv2.circle(frame, (predicted_x, predicted_y), 5, (0, 0, 255), -1)
        cv2.putText(frame, "Tracking", (predicted_x, predicted_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Display the frame
    cv2.imshow('Real-Time Face Detection & Tracking', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


No LBPH model found. Only detection will work.


In [ ]:
import cv2
import numpy as np

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Initialize OpenCV's HOG + SVM face detector
hog_face_detector = cv2.HOGDescriptor()
hog_face_detector.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Initialize Local Binary Pattern Histogram (LBPH) Face Recognizer
lbph_recognizer = cv2.face.LBPHFaceRecognizer_create()

# Load pre-trained LBPH model (if available)
try:
    lbph_recognizer.read("lbph_face_model.xml")  # Ensure you have a trained model
    print("LBPH model loaded successfully.")
except:
    print("No LBPH model found. Only detection will work.")

# Variables for tracking
tracking = False
tracked_points = None
old_gray = None
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # HOG-SVM Face Detection
    faces, _ = hog_face_detector.detectMultiScale(gray, winStride=(8, 8), padding=(8, 8), scale=1.05)

    # If a face is detected, start tracking or reinitialize tracking
    if len(faces) > 0:
        x, y, w, h = faces[0]  # Take the first detected face

        # Initialize tracking points (corners of the bounding box)
        tracked_points = np.array([[x, y], [x + w, y], [x, y + h], [x + w, y + h]], dtype=np.float32).reshape(-1, 1, 2)
        old_gray = gray.copy()  # Save current frame for optical flow
        tracking = True

        # Recognize face with LBPH (if trained)
        face_roi = gray[y:y+h, x:x+w]
        try:
            label, confidence = lbph_recognizer.predict(face_roi)
            text = f"ID: {label} | Conf: {round(confidence, 2)}"
            cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        except:
            pass  # If no trained model, skip recognition

        # Draw bounding box around the detected face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    elif tracking and tracked_points is not None:
        # Calculate optical flow (i.e., track points)
        next_points, status, _ = cv2.calcOpticalFlowPyrLK(old_gray, gray, tracked_points, None, **lk_params)

        # Filter out bad points
        good_new = next_points[status == 1]
        good_old = tracked_points[status == 1]

        # Compute bounding box from the tracked points
        if len(good_new) > 0:
            x_min = int(np.min(good_new[:, 0, 0]))
            y_min = int(np.min(good_new[:, 0, 1]))
            x_max = int(np.max(good_new[:, 0, 0]))
            y_max = int(np.max(good_new[:, 0, 1]))
            
            # Update the bounding box with the new coordinates
            tracked_bbox = (x_min, y_min, x_max - x_min, y_max - y_min)

            # Draw the new bounding box
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

            # Update the previous frame and previous points
            old_gray = gray.copy()
            tracked_points = good_new.reshape(-1, 1, 2)

        else:
            # If tracking failed, stop tracking
            tracking = False
            tracked_points = None
            print("Lost track of face.")

    # Display the frame
    cv2.imshow('Real-Time Face Detection & Tracking', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


No LBPH model found. Only detection will work.


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [ ]:
import os
import cv2
import dlib
import numpy as np
import time # For FPS calculation (optional)

# --- Configuration ---
# Main dataset folder containing subfolders named after people
# Example structure:
# /path/to/your/dataset/
#  |- Person_A/
#  |  |- img1.jpg
#  |  |- img2.png
#  |- Person_B/
#  |  |- pic1.jpeg
# ...
dataset_path = "dataset/dataset" 

# Size to resize cropped faces to (consistency is key for LBPH)
face_size = (100, 100)

# LBPH Confidence threshold (lower values mean higher confidence)
# Adjust this based on testing - might need values between 50 and 100
confidence_threshold = 25

# --- Global Variables ---
# Load Dlib's pre-trained frontal face detector (HOG-based)
detector = dlib.get_frontal_face_detector()

# Create LBPH Face Recognizer
# Note: You might need to install opencv-contrib-python
# pip install opencv-contrib-python
recognizer = cv2.face.LBPHFaceRecognizer_create()

# Dictionary to map numerical labels to names
label_to_name = {}

# --- Function to Prepare Training Data and Train LBPH ---
def train_model(data_folder_path):
    """
    Scans the dataset folder, detects faces, prepares training data,
    and trains the LBPH recognizer.
    """
    print(f"[INFO] Preparing training data from: {data_folder_path}")
    faces = []
    labels = []
    current_label_id = 0
    global label_to_name # Allow modification of the global dictionary

    # Iterate through each person's folder in the dataset path
    for person_name in os.listdir(data_folder_path):
        person_folder_path = os.path.join(data_folder_path, person_name)

        # Skip if it's not a directory
        if not os.path.isdir(person_folder_path):
            continue

        print(f"[INFO] Processing images for: {person_name}")
        # Assign a numerical label to this person if not already done
        if person_name not in label_to_name.values():
            label_to_name[current_label_id] = person_name
            person_label = current_label_id
            current_label_id += 1
        else:
            # Find existing label ID if name already exists (shouldn't happen with unique folder names)
            person_label = [id for id, name in label_to_name.items() if name == person_name][0]


        # Iterate through images in the person's folder
        for image_name in os.listdir(person_folder_path):
            image_path = os.path.join(person_folder_path, image_name)

            # Read the image
            image = cv2.imread(image_path)
            if image is None:
                print(f"[WARNING] Could not read image: {image_path}. Skipping.")
                continue

            # Convert to grayscale (Dlib detector and LBPH work better with grayscale)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # Optional: Histogram Equalization can sometimes improve detection/recognition
            # gray = cv2.equalizeHist(gray)

            # Detect faces using Dlib detector
            # The '1' indicates upsampling the image 1 time, making it larger
            # and potentially detecting more faces, but it's slower. Use 0 for faster.
            detected_faces = detector(gray, 1)

            if len(detected_faces) == 0:
                print(f"[WARNING] No face detected in: {image_path}. Skipping.")
                continue

            # Process only the first detected face for simplicity in training
            face_rect = detected_faces[0]
            x, y, w, h = face_rect.left(), face_rect.top(), face_rect.width(), face_rect.height()

            # Ensure coordinates are valid
            if x < 0 or y < 0 or w <= 0 or h <= 0:
                print(f"[WARNING] Invalid face bounds detected in: {image_path}. Skipping.")
                continue

            # Crop the face region from the grayscale image
            face_roi = gray[y:y+h, x:x+w]

            # Check if cropping resulted in an empty image
            if face_roi.size == 0:
                 print(f"[WARNING] Empty face ROI after cropping in: {image_path}. Skipping.")
                 continue

            # Resize the face ROI to the standard size
            resized_face = cv2.resize(face_roi, face_size, interpolation=cv2.INTER_AREA)

            # Append the processed face and its corresponding label
            faces.append(resized_face)
            labels.append(person_label)

    if not faces:
        print("[ERROR] No faces found for training. Check dataset path and image content.")
        return False

    print(f"[INFO] Total faces prepared for training: {len(faces)}")
    print(f"[INFO] Total unique persons (labels): {len(label_to_name)}")
    print("[INFO] Training LBPH model...")

    # Train the recognizer
    # Ensure labels is a NumPy array of type int32
    recognizer.train(faces, np.array(labels, dtype=np.int32))

    print("[INFO] LBPH model trained successfully.")
    # Optional: Save the trained model and label mapping for later use
    # recognizer.save("lbph_model.yml")
    # with open("label_map.txt", "w") as f:
    #    for label_id, name in label_to_name.items():
    #        f.write(f"{label_id}:{name}\n")
    # print("[INFO] Model and label map saved.")
    return True

# --- Function for Live Face Recognition ---
def run_live_recognition():
    """
    Opens the camera, detects faces, and performs recognition using the trained model.
    """
    global label_to_name # Access the global mapping

    print("[INFO] Starting video stream...")
    # Use 0 for default camera, or change if you have multiple cameras
    cap = cv2.VideoCapture(0)
    time.sleep(1.0) # Allow camera sensor to warm up

    if not cap.isOpened():
        print("[ERROR] Cannot open camera. Exiting.")
        return

    prev_time = 0

    while True:
        # Read frame from camera
        ret, frame = cap.read()
        if not ret or frame is None:
            print("[ERROR] Failed to grab frame. Exiting.")
            break

        # For FPS calculation
        current_time = time.time()
        fps = 1 / (current_time - prev_time)
        prev_time = current_time

        # Convert frame to grayscale for detection
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Optional: Equalize histogram
        # gray_frame = cv2.equalizeHist(gray_frame)

        # Detect faces in the current frame
        detected_faces = detector(gray_frame, 0) # Use 0 for speed in live feed

        # Loop over detected faces
        for face_rect in detected_faces:
            x, y, w, h = face_rect.left(), face_rect.top(), face_rect.width(), face_rect.height()

            # Ensure coordinates are valid and within frame boundaries
            x = max(0, x)
            y = max(0, y)
            w = min(w, frame.shape[1] - x)
            h = min(h, frame.shape[0] - y)

            if w <= 0 or h <= 0:
                continue # Skip invalid detections

            # Crop the face region *from the grayscale frame* for recognition
            face_roi = gray_frame[y:y+h, x:x+w]

             # Check if cropping resulted in an empty image
            if face_roi.size == 0:
                 continue

            # Resize the cropped face to the standard size used for training
            resized_face = cv2.resize(face_roi, face_size, interpolation=cv2.INTER_AREA)

            # Perform prediction using the trained LBPH recognizer
            label_id, confidence = recognizer.predict(resized_face)

            # Get the name associated with the predicted label ID
            # Use "Unknown" if confidence is too high (poor match) or label unknown
            if confidence < confidence_threshold and label_id in label_to_name:
                name = label_to_name[label_id]
                display_text = f"{name} ({confidence:.2f})"
            else:
                name = "Unknown"
                display_text = f"{name} ({confidence:.2f})" # Still show confidence for unknowns

            # Draw bounding box around the face on the *original color* frame
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Put the predicted name (and confidence) text above the bounding box
            text_y = y - 10 if y - 10 > 10 else y + 10 # Position text above box
            cv2.putText(frame, display_text, (x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Display FPS on frame (optional)
        cv2.putText(frame, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Show the resulting frame
        cv2.imshow("Live Face Recognition (Press 'q' to quit)", frame)

        # Check for exit key ('q')
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # --- Cleanup ---
    print("[INFO] Stopping video stream and cleaning up...")
    cap.release()
    cv2.destroyAllWindows()
    print("[INFO] Application finished.")

# --- Main Execution ---
if __name__ == "__main__":
    if not os.path.isdir(dataset_path):
         print(f"[ERROR] Dataset path not found or is not a directory: {dataset_path}")
         print("[INFO] Please set the 'dataset_path' variable correctly.")
    elif train_model(dataset_path): # Train the model first
        run_live_recognition() # If training is successful, run live recognition
    else:
        print("[ERROR] Model training failed. Cannot proceed to live recognition.")

[INFO] Preparing training data from: dataset/dataset
[INFO] Processing images for: amisha
[WARNING] No face detected in: dataset/dataset\amisha\ezgif-frame-030.jpg. Skipping.
[WARNING] No face detected in: dataset/dataset\amisha\ezgif-frame-040.jpg. Skipping.
[WARNING] No face detected in: dataset/dataset\amisha\IMG-20250331-WA0023.jpg. Skipping.
[WARNING] No face detected in: dataset/dataset\amisha\IMG-20250331-WA0028.jpg. Skipping.
[INFO] Processing images for: dhanoosh
[WARNING] No face detected in: dataset/dataset\dhanoosh\ezgif-frame-060.jpg. Skipping.
[WARNING] No face detected in: dataset/dataset\dhanoosh\ezgif-frame-130.jpg. Skipping.
[INFO] Processing images for: jose
[WARNING] No face detected in: dataset/dataset\jose\ezgif-frame-005.jpg. Skipping.
[WARNING] No face detected in: dataset/dataset\jose\ezgif-frame-015.jpg. Skipping.
[WARNING] No face detected in: dataset/dataset\jose\ezgif-frame-035.jpg. Skipping.
[WARNING] No face detected in: dataset/dataset\jose\IMG-20250331-

In [ ]:
import os
import cv2
import dlib
import numpy as np
import time # For FPS calculation (optional)

# --- Configuration ---
# Main dataset folder containing subfolders named after people
# Example structure:
# /path/to/your/dataset/
#  |- Person_A/
#  |  |- img1.jpg
#  |  |- img2.png
#  |- Person_B/
#  |  |- pic1.jpeg
# ...
dataset_path = "dataset/dataset"

# Size to resize cropped faces to (consistency is key for LBPH)
face_size = (100, 100)

# LBPH Confidence threshold (lower values mean higher confidence)
# Adjust this based on testing - might need values between 50 and 100
confidence_threshold = 50 # Adjusted slightly for potentially better testing results

# --- NEW: Testing Configuration ---
# Set to True to test the model on the dataset after training
activate_test = True

# --- Global Variables ---
# Load Dlib's pre-trained frontal face detector (HOG-based)
detector = dlib.get_frontal_face_detector()

# Create LBPH Face Recognizer
# Note: You might need to install opencv-contrib-python
# pip install opencv-contrib-python
recognizer = cv2.face.LBPHFaceRecognizer_create()

# Dictionary to map numerical labels to names
label_to_name = {}

# --- Function to Prepare Training Data and Train LBPH ---
def train_model(data_folder_path):
    """
    Scans the dataset folder, detects faces, prepares training data,
    and trains the LBPH recognizer.
    Returns True if training was successful, False otherwise.
    """
    print(f"[INFO] Preparing training data from: {data_folder_path}")
    faces = []
    labels = []
    current_label_id = 0
    global label_to_name # Allow modification of the global dictionary
    label_to_name.clear() # Clear mapping for potential re-training runs

    # Iterate through each person's folder in the dataset path
    for person_name in os.listdir(data_folder_path):
        person_folder_path = os.path.join(data_folder_path, person_name)

        # Skip if it's not a directory
        if not os.path.isdir(person_folder_path):
            continue

        print(f"[INFO] Processing images for: {person_name}")
        # Assign a numerical label to this person
        if person_name not in label_to_name.values():
            label_to_name[current_label_id] = person_name
            person_label = current_label_id
            current_label_id += 1
        else:
            # This case should ideally not happen if folder names are unique identifiers
            person_label = [id for id, name in label_to_name.items() if name == person_name][0]


        # Iterate through images in the person's folder
        image_count = 0
        for image_name in os.listdir(person_folder_path):
            image_path = os.path.join(person_folder_path, image_name)

            # Read the image
            image = cv2.imread(image_path)
            if image is None:
                print(f"[WARNING] Could not read image: {image_path}. Skipping.")
                continue

            # Convert to grayscale (Dlib detector and LBPH work better with grayscale)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # Optional: Histogram Equalization can sometimes improve detection/recognition
            # gray = cv2.equalizeHist(gray)

            # Detect faces using Dlib detector
            # Using upsampling=1 consistent with potential quality needs during training data prep
            detected_faces = detector(gray, 1)

            if len(detected_faces) == 0:
                print(f"[WARNING] No face detected in: {image_path} for {person_name}. Skipping.")
                continue
            elif len(detected_faces) > 1:
                 print(f"[WARNING] Multiple faces ({len(detected_faces)}) detected in: {image_path} for {person_name}. Using the first one.")

            # Process only the first detected face for consistency
            face_rect = detected_faces[0]
            x, y, w, h = face_rect.left(), face_rect.top(), face_rect.width(), face_rect.height()

            # Ensure coordinates are valid
            x = max(0, x)
            y = max(0, y)
            if w <= 0 or h <= 0 or x + w > gray.shape[1] or y + h > gray.shape[0]:
                print(f"[WARNING] Invalid face bounds detected or calculated in: {image_path}. Skipping.")
                continue

            # Crop the face region from the grayscale image
            face_roi = gray[y:y+h, x:x+w]

            # Check if cropping resulted in an empty image (due to edge cases)
            if face_roi.size == 0:
                 print(f"[WARNING] Empty face ROI after cropping in: {image_path}. Skipping.")
                 continue

            # Resize the face ROI to the standard size
            try:
                resized_face = cv2.resize(face_roi, face_size, interpolation=cv2.INTER_AREA)
            except cv2.error as e:
                 print(f"[WARNING] Error resizing face ROI from {image_path}: {e}. Skipping.")
                 continue


            # Append the processed face and its corresponding label
            faces.append(resized_face)
            labels.append(person_label)
            image_count += 1

        print(f"[INFO] Added {image_count} face images for {person_name}.")


    if not faces:
        print("[ERROR] No faces found or processed for training. Check dataset path, image content, and warnings.")
        return False

    if len(label_to_name) < 2:
         print("[WARNING] Training requires images from at least two different people for the recognizer to be effective.")
         # Decide if you want to stop here or proceed with a potentially less useful model
         # return False # Option to stop if less than 2 people

    print(f"\n[INFO] Total faces prepared for training: {len(faces)}")
    print(f"[INFO] Total unique persons (labels): {len(label_to_name)}")
    print(f"[INFO] Label map: {label_to_name}")
    print("[INFO] Training LBPH model...")

    # Train the recognizer
    # Ensure labels is a NumPy array of type int32
    recognizer.train(faces, np.array(labels, dtype=np.int32))

    print("[INFO] LBPH model trained successfully.")
    # Optional: Save the trained model and label mapping for later use
    # recognizer.save("lbph_model.yml")
    # with open("label_map.txt", "w") as f:
    #  for label_id, name in label_to_name.items():
    #     f.write(f"{label_id}:{name}\n")
    # print("[INFO] Model and label map saved.")
    return True

# --- NEW: Function to Test Model on Dataset ---
def test_model_on_dataset(data_folder_path):
    """
    Tests the trained LBPH model on the images in the dataset folder
    and calculates the success rate.
    """
    print("\n" + "="*30)
    print("[INFO] Starting model testing on the dataset...")
    print("="*30)

    global label_to_name, recognizer, detector, face_size, confidence_threshold

    if not label_to_name:
        print("[ERROR] Label map is empty. Cannot perform testing. Was the model trained?")
        return

    # Create a reverse map for convenience (Name -> Label ID)
    name_to_label = {name: label_id for label_id, name in label_to_name.items()}

    total_tested_faces = 0
    correct_predictions = 0
    faces_detected_count = 0

    # Iterate through each person's folder in the dataset path
    for person_name in os.listdir(data_folder_path):
        person_folder_path = os.path.join(data_folder_path, person_name)

        if not os.path.isdir(person_folder_path):
            continue

        # Check if this person was part of the training set
        if person_name not in name_to_label:
            print(f"[WARNING] Person '{person_name}' found in dataset but not in the trained label map. Skipping testing for this person.")
            continue

        true_label_id = name_to_label[person_name]
        print(f"[TESTING] Evaluating images for: {person_name} (Expected Label: {true_label_id})")

        images_in_folder = 0
        detections_in_folder = 0
        correct_in_folder = 0

        # Iterate through images in the person's folder
        for image_name in os.listdir(person_folder_path):
            image_path = os.path.join(person_folder_path, image_name)
            images_in_folder += 1

            image = cv2.imread(image_path)
            if image is None:
                # Warning already given during training, maybe skip here unless verbose testing needed
                continue

            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # gray = cv2.equalizeHist(gray) # Apply if used in training

            # Use the same detection settings as training ideally
            detected_faces = detector(gray, 1)

            if len(detected_faces) == 0:
                 # Can log this if needed: print(f"[TESTING-WARN] No face detected in: {image_path}")
                 continue # Cannot test recognition if no face is found

            faces_detected_count += 1
            detections_in_folder += 1

            # Use only the first detected face for consistency with training
            face_rect = detected_faces[0]
            x, y, w, h = face_rect.left(), face_rect.top(), face_rect.width(), face_rect.height()

            x = max(0, x)
            y = max(0, y)
            if w <= 0 or h <= 0 or x + w > gray.shape[1] or y + h > gray.shape[0]:
                continue # Skip invalid bounds

            face_roi = gray[y:y+h, x:x+w]
            if face_roi.size == 0:
                 continue # Skip empty ROI

            try:
                resized_face = cv2.resize(face_roi, face_size, interpolation=cv2.INTER_AREA)
            except cv2.error:
                continue # Skip resize errors


            # Perform prediction
            predicted_label_id, confidence = recognizer.predict(resized_face)

            total_tested_faces += 1 # Count this face as tested for recognition

            # Evaluate prediction
            prediction_is_correct = False
            if confidence < confidence_threshold and predicted_label_id == true_label_id:
                correct_predictions += 1
                correct_in_folder += 1
                prediction_is_correct = True

            # Optional: Print detailed results per image (can be very verbose)
            # predicted_name = label_to_name.get(predicted_label_id, "Unknown Label")
            # print(f"  - {image_name}: Predicted={predicted_name}({predicted_label_id}), Confidence={confidence:.2f}, Correct={prediction_is_correct}")

        print(f"[TESTING] Finished {person_name}: Found faces in {detections_in_folder}/{images_in_folder} images. Correctly recognized: {correct_in_folder}/{detections_in_folder}")


    print("\n" + "="*30)
    print("[INFO] Dataset Testing Summary")
    print("="*30)
    print(f"Total faces detected across all tested images: {faces_detected_count}")
    print(f"Total detected faces subjected to recognition: {total_tested_faces}")
    print(f"Total correct recognitions (within threshold): {correct_predictions}")

    if total_tested_faces > 0:
        accuracy = (correct_predictions / total_tested_faces) * 100
        print(f"Recognition Success Rate: {accuracy:.2f}%")
    else:
        print("Recognition Success Rate: N/A (No faces were successfully processed for recognition testing)")
    print("="*30 + "\n")


# --- Function for Live Face Recognition ---
def run_live_recognition():
    """
    Opens the camera, detects faces, and performs recognition using the trained model.
    """
    global label_to_name # Access the global mapping

    print("[INFO] Starting video stream for live recognition...")
    # Use 0 for default camera, or change if you have multiple cameras
    cap = cv2.VideoCapture(0)
    time.sleep(1.0) # Allow camera sensor to warm up

    if not cap.isOpened():
        print("[ERROR] Cannot open camera. Exiting.")
        return

    prev_time = 0

    while True:
        # Read frame from camera
        ret, frame = cap.read()
        if not ret or frame is None:
            print("[ERROR] Failed to grab frame. Exiting.")
            break

        # For FPS calculation
        current_time = time.time()
        # Avoid division by zero on the first frame
        time_diff = current_time - prev_time
        fps = 1 / time_diff if time_diff > 0 else 0
        prev_time = current_time

        # Convert frame to grayscale for detection
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Optional: Equalize histogram
        # gray_frame = cv2.equalizeHist(gray_frame)

        # Detect faces in the current frame
        # Using upsampling=0 for potentially faster live performance
        detected_faces = detector(gray_frame, 0)

        # Loop over detected faces
        for face_rect in detected_faces:
            x, y, w, h = face_rect.left(), face_rect.top(), face_rect.width(), face_rect.height()

            # Ensure coordinates are valid and within frame boundaries
            x = max(0, x)
            y = max(0, y)
            # Check width/height before calculating bottom-right corner
            if w <= 0 or h <= 0:
                continue
            # Adjust width/height if they extend beyond frame boundaries
            w = min(w, frame.shape[1] - x)
            h = min(h, frame.shape[0] - y)
            # Re-check after adjustment
            if w <= 0 or h <= 0:
                continue


            # Crop the face region *from the grayscale frame* for recognition
            face_roi = gray_frame[y:y+h, x:x+w]

             # Check if cropping resulted in an empty image (should be rare with checks above)
            if face_roi.size == 0:
                continue

            # Resize the cropped face to the standard size used for training
            try:
                resized_face = cv2.resize(face_roi, face_size, interpolation=cv2.INTER_AREA)
            except cv2.error:
                continue # Skip if resize fails


            # Perform prediction using the trained LBPH recognizer
            label_id, confidence = recognizer.predict(resized_face)

            # Get the name associated with the predicted label ID
            # Use "Unknown" if confidence is too high (poor match) or label unknown
            name = "Unknown" # Default
            print(confidence)
            if label_id in label_to_name: # Check if label exists first
                 if confidence > confidence_threshold:
                      name = label_to_name[label_id]
                      display_text = f"{name} ({confidence:.2f})"
                 else:
                      # Known person, but low confidence
                      display_text = f"Maybe {label_to_name[label_id]}? ({confidence:.2f})"
                      # Or just stick with "Unknown":
                      # name = "Unknown"
                      # display_text = f"{name} ({confidence:.2f})"
            else:
                 # Label ID not in our map (shouldn't happen if predict works correctly)
                 display_text = f"Unknown Label {label_id} ({confidence:.2f})"


            # Draw bounding box around the face on the *original color* frame
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Put the predicted name (and confidence) text above the bounding box
            text_y = y - 10 if y - 10 > 10 else y + h + 20 # Adjust position if too close to top
            cv2.putText(frame, display_text, (x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Display FPS on frame (optional)
        cv2.putText(frame, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Show the resulting frame
        cv2.imshow("Live Face Recognition (Press 'q' to quit)", frame)

        # Check for exit key ('q')
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # --- Cleanup ---
    print("[INFO] Stopping video stream and cleaning up...")
    cap.release()
    cv2.destroyAllWindows()
    print("[INFO] Application finished.")


# --- Main Execution ---
if __name__ == "__main__":
    if not os.path.isdir(dataset_path):
         print(f"[ERROR] Dataset path not found or is not a directory: {dataset_path}")
         print("[INFO] Please set the 'dataset_path' variable correctly.")
    else:
        # 1. Train the model
        training_successful = train_model(dataset_path)

        if training_successful:
            # 2. Test the model (if activated)
            if activate_test:
                test_model_on_dataset(dataset_path)
            else:
                 print("[INFO] Skipping dataset testing as 'activate_test' is False.")

            # 3. Run live recognition (only if training was successful)
            # Decide if you always want to run live, or maybe only if not testing, etc.
            # Current logic: Run live recognition after training and optional testing.
            run_live_recognition()
        else:
            print("[ERROR] Model training failed. Cannot proceed to testing or live recognition.")
            

[INFO] Preparing training data from: dataset/dataset
[INFO] Processing images for: amisha
[WARNING] No face detected in: dataset/dataset\amisha\ezgif-frame-030.jpg for amisha. Skipping.
[WARNING] No face detected in: dataset/dataset\amisha\ezgif-frame-040.jpg for amisha. Skipping.
[WARNING] No face detected in: dataset/dataset\amisha\IMG-20250331-WA0023.jpg for amisha. Skipping.
[WARNING] No face detected in: dataset/dataset\amisha\IMG-20250331-WA0028.jpg for amisha. Skipping.
[INFO] Added 17 face images for amisha.
[INFO] Processing images for: dhanoosh
[WARNING] No face detected in: dataset/dataset\dhanoosh\ezgif-frame-058.png for dhanoosh. Skipping.
[WARNING] No face detected in: dataset/dataset\dhanoosh\ezgif-frame-059.png for dhanoosh. Skipping.
[WARNING] No face detected in: dataset/dataset\dhanoosh\ezgif-frame-060.png for dhanoosh. Skipping.
[WARNING] No face detected in: dataset/dataset\dhanoosh\ezgif-frame-061.png for dhanoosh. Skipping.
[WARNING] No face detected in: dataset/